In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec

2023-04-30 12:49:29.662869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the dataset
df = pd.read_csv("../../data/cleaned/out.csv")

# Text preprocessing
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text

df['text'] = df['text'].apply(preprocess_text)
df['text'].head()

0     tiffanylue i know i was listenin to bad habit...
1    layin n bed with a headache ughhhh waitin on y...
2                      funeral ceremony gloomy friday 
3                 wants to hang out with friends soon 
4     dannycastillo we want to trade with someone w...
Name: text, dtype: object

In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(remove_stopwords)
df['text'].head()

[nltk_data] Downloading package stopwords to /home/juan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    tiffanylue know listenin bad habit earlier sta...
1              layin n bed headache ughhhh waitin call
2                       funeral ceremony gloomy friday
3                              wants hang friends soon
4    dannycastillo want trade someone houston ticke...
Name: text, dtype: object

In [4]:
tokenized_texts = [text.split() for text in df['text']]
word2vec = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1)

In [5]:
def text_to_vector(text):
    words = text.split()
    word_vectors = [word2vec.wv[word] for word in words if word in word2vec.wv]
    if not word_vectors:
        return np.zeros(word2vec.vector_size).tolist()
    return np.mean(word_vectors, axis=0).tolist()


df['vector'] = df['text'].apply(text_to_vector)

In [6]:
encoder = OneHotEncoder(sparse=False)
encoded_labels = encoder.fit_transform(df['label'].values.reshape(-1, 1))


/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
encoded_labels.shape

(65989, 7)

In [8]:
encoded_labels[0]

array([1., 0., 0., 0., 0., 0., 0.])

In [9]:
df.head()

,label,text,vector
0,0,tiffanylue know listenin bad habit earlier sta...,"[-0.1719031035900116, 0.4805227518081665, -0.1..."
1,0,layin n bed headache ughhhh waitin call,"[-0.3772401511669159, 0.4716305136680603, -0.0..."
2,0,funeral ceremony gloomy friday,"[-0.28188958764076233, 0.25577205419540405, -0..."
3,1,wants hang friends soon,"[-0.4622727632522583, 0.8539247512817383, -0.1..."
4,6,dannycastillo want trade someone houston ticke...,"[-0.228619784116745, 0.6895619630813599, -0.04..."


In [10]:
def convert_to_tensor(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float32)
    return arg

In [11]:
X = np.array([np.array(vec) for vec in df['vector'].to_numpy()], dtype=np.float32)
y = encoded_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
type(X_train[0])

numpy.ndarray

In [14]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_train shape: (52791, 100)
y_train shape: (52791, 7)


In [15]:
# Model parameters
input_dim = 100  # Word2Vec vector size
output_dim = 7   # Number of sentiment labels
hidden_units = 32  # LSTM hidden units

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=input_dim+1, output_dim=input_dim, input_length=input_dim))
model.add(LSTM(hidden_units))
model.add(Dense(output_dim, activation='softmax'))
model.summary()
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          10100     
                                                                 
 lstm_1 (LSTM)               (None, 32)                17024     
                                                                 
 dense_1 (Dense)             (None, 7)                 231       
                                                                 
Total params: 27,355
Trainable params: 27,355
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/embedding_1/embedding_lookup' defined at (most recent call last):
    File "/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215